In [32]:
#Import Libraries needed for API, and Pandas
import requests
import pandas as pd
import numpy as np
import ConfigParser

#Read config file with Looker API and Database connection information
config = ConfigParser.RawConfigParser(allow_no_value=True)
config.read('config')

#Very Basic Looker API class allowing us to access the data from a given Look ID
class lookerAPIClient:
    def __init__(self, api_host=None, api_client_id=None, api_secret=None, api_port='19999'):
        auth_request_payload = {'client_id': api_client_id, 'client_secret': api_secret}
        self.host = api_host
        self.uri_stub = '/api/3.0/'
        self.uri_full = ''.join([api_host, ':', api_port, self.uri_stub])
        response = requests.post(self.uri_full + 'login', params=auth_request_payload)
        authData = response.json()
        self.access_token = authData['access_token']
        self.auth_headers = {
                'Authorization' : 'token ' + self.access_token,
                }

    def post(self, call='', json_payload=None):
        response = requests.post(self.uri_full + call, headers=self.auth_headers, json=json_payload)
        return response.json()

    def get(self, call=''):
        response = requests.get(self.uri_full + call, headers=self.auth_headers)
        return response.json()

    def runLook(self, look, limit=100):
        optional_arguments = '?' + 'limit=' + str(limit)
        return self.get('/'.join(['looks',look,'run','json'])+optional_arguments)

#Initialize the Looker API Class with the data in our config file (which is stored in a neighboring file 'config')
x = lookerAPIClient(
        api_host      = config.get('api', 'api_host'), 
        api_client_id = config.get('api', 'api_client_id'), 
        api_secret    = config.get('api', 'api_secret'), 
        api_port      = config.get('api', 'api_port')
        )    
    
#Use the API to get our training/'test' dataset and our new 'validation' dataset we will predict upon
historicalCustomers = x.runLook('292',limit=10000)
newCustomers = x.runLook('293',limit=10000)


historicalCustomersDF = pd.DataFrame(historicalCustomers)
newCustomersDF  = pd.DataFrame(newCustomers)


In [33]:
import statsmodels.api as sm
# import pandas.tseries.statsmodels.api as sm
trainingSet.head()

Y = historicalCustomersDF['user_facts.total_revenue']
X = historicalCustomersDF[['users.age','user_facts.orders_in_first_30_days','user_facts.total_revenue_in_first_30_days']]

X = sm.add_constant(X)

est = sm.OLS(Y,X)

In [34]:
est = est.fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     user_facts.total_revenue   R-squared:                       0.138
Model:                                  OLS   Adj. R-squared:                  0.138
Method:                       Least Squares   F-statistic:                     203.3
Date:                      Sat, 09 Sep 2017   Prob (F-statistic):          2.64e-122
Time:                              21:07:32   Log-Likelihood:                -25629.
No. Observations:                      3808   AIC:                         5.127e+04
Df Residuals:                          3804   BIC:                         5.129e+04
Df Model:                                 3                                         
Covariance Type:                  nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                       200.5521      8.993     22.300      0.000     182.920     218.184
users.age                                    -0.3200      0.175     -1.824      0.068      -0.664       0.024
user_facts.orders_in_first_30_days          -82.1189      7.208    -11.393      0.000     -96.250     -67.988
user_facts.total_revenue_in_first_30_days     1.1115      0.047     23.791      0.000       1.020       1.203
==============================================================================
Omnibus:                     2142.726   Durbin-Watson:                   0.951
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18301.039
Skew:                           2.591   Prob(JB):                         0.00
Kurtosis:                      12.407   Cond. No.                         318.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
## Validation Set Shaping 
X2 = newCustomersDF[['users.age','user_facts.orders_in_first_30_days', 'user_facts.total_revenue_in_first_30_days']]
X2 = sm.add_constant(X2)
## END Validation Set Shaping 

output = pd.concat([newCustomersDF[['users.id']],est.predict(X2)],axis=1)

In [36]:
#from __future__ import print_function
from datetime import date, datetime, timedelta
import mysql.connector

cnx = mysql.connector.connect(
                              user     = config.get('database', 'user'), 
                              password = config.get('database', 'password'),
                              host     = config.get('database', 'host'),
                              database = config.get('database', 'database')
                             )
cursor = cnx.cursor()

cursor.execute('truncate table my_schema.ltv_predictions')

for elem in output.itertuples():
    add_record = ("INSERT INTO my_schema.ltv_predictions (user_id, ltv_prediction) VALUES (%s, %s)")
    cursor.execute(add_record,(str(elem[1]),str(elem[2])))


cnx.commit()
cursor.close()
cnx.close()
